## Huỳnh Đức Tính - SE171120

In [8]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
import csv, sqlite3

con = sqlite3.connect("SQLiteMagic.db")
cur = con.cursor()

In [10]:
!pip install -q pandas==1.1.5

In [11]:
import pandas as pd
df = pd.read_csv("Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv")
df.to_sql("ChicagoCensus",con, if_exists = 'replace', index = False, method = 'multi')

df = pd.read_csv("ChicagoCrimeData.csv")
df.to_sql("ChicagoCrime",con, if_exists = 'replace', index = False, method = 'multi')

df = pd.read_csv('Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012_.csv')
df.to_sql("ChicagoPublicSchools", con, if_exists = 'replace', index = False, method = 'multi')

566

In [12]:
%sql sqlite:///SQLiteMagic.db

### Problem 1: Find the total number of crimes recorded in the CRIME table.


In [19]:
%%sql
Select count(*) as "Number of Crimes" 
From ChicagoCrime

 * sqlite:///SQLiteMagic.db
Done.


Number of Crimes
533


### Problem 2: List community areas with per capita income less than 11000.


In [18]:
%%sql
Select "COMMUNITY AREA NAME", "PER CAPITA INCOME "
From ChicagoCensus
Where "PER CAPITA INCOME " < 11000

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY AREA NAME,PER CAPITA INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3: List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [12]:
%%sql 
Select case_number 
From ChicagoCrime 
Where (lower(DESCRIPTION) Like "%minor%") 

 * sqlite:///SQLiteMagic.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4: List all kidnapping crimes involving a child?


In [13]:
%%sql
Select *
From ChicagoCrime
Where lower(Primary_Type) = 'kidnapping'

 * sqlite:///SQLiteMagic.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5: What kinds of crimes were recorded at schools?


In [14]:
%%sql
Select distinct(Primary_type)
From ChicagoCrime
Where LOCATION_DESCRIPTION Like "%SCHOOL%"

 * sqlite:///SQLiteMagic.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6: List the average safety score for each type of school.


In [15]:
%%sql
Select "Elementary, Middle, or High School", avg("Safety Score") as "Average Safety Score"
From ChicagoPublicSchools
Group by "Elementary, Middle, or High School"

 * sqlite:///SQLiteMagic.db
Done.


"Elementary, Middle, or High School",Average Safety Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7: List 5 community areas with highest % of households below poverty line


In [16]:
%%sql
Select "Community area name", "PERCENT HOUSEHOLDS BELOW POVERTY"
From ChicagoCensus
Order By "PERCENT HOUSEHOLDS BELOW POVERTY" DESC Limit 5;

 * sqlite:///SQLiteMagic.db
Done.


COMMUNITY AREA NAME,PERCENT HOUSEHOLDS BELOW POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8: Which community area is most crime prone?


In [47]:
%%sql
Select "Community Area Number", max("Frequency") as "Frequency"
From (Select Cen."Community Area Number",Cen."Community Area Name", count(Crime."Case_number") as "Frequency"
      From ChicagoCrime Crime, ChicagoCensus Cen
      Where Cen."Community Area Number" = Crime."Community_Area_Number"
      Group by Cen."Community Area Name"
      Order By "Frequency" DESC)

 * sqlite:///SQLiteMagic.db
Done.


Community Area Number,Frequency
25.0,43


### Problem 9: Use a sub-query to find the name of the community area with highest hardship index


In [48]:
%%sql
Select "Community Area Name", max("hardship index") as "max of Harship_index"
From (Select "Community Area Name", "hardship index"
      From ChicagoCensus
      Order By "hardship index" DESC)

 * sqlite:///SQLiteMagic.db
Done.


Community Area Name,max of Harship_index
Riverdale,98.0


### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?


In [49]:
%%sql
Select "Community Area Name", max("Frequency") as "Frequency"
From (Select Cen."Community Area Number",Cen."Community Area Name", count(Crime."Case_number") as "Frequency"
      From ChicagoCrime Crime, ChicagoCensus Cen
      Where Cen."Community Area Number" = Crime."Community_Area_Number"
      Group by Cen."Community Area Name"
      Order By "Frequency" DESC)

 * sqlite:///SQLiteMagic.db
Done.


Community Area Name,Frequency
Austin,43
